<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">


# Derivatives Analytics with Python
**&mdash;Part IIIb: Market-Based Valuation (Ch.&nbsp;11&ndash;13)**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>


## Part IIIb &mdash; Market-Based Valuation

### Chapter 12 &mdash; Valuation in the Calibrated Model

This notebook studies valuation tasks in the calibrated BCC model.

- Simulate short-rate, variance, and index level paths under BCC dynamics.
- Value European index options via Monte Carlo simulation.
- Compare Monte Carlo prices to Fourier-based benchmark values.
- Implement Least-Squares Monte Carlo valuation for American puts.

Reference scripts from the original companion repository are located under
`x_store/dawp/python36/12_val`.


### 1. Environment and figure configuration

We start by importing the core libraries for this class, printing version
information, and configuring the figure export helper used to generate
PDFs for the slide deck.


In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for data and figure export

import math  # elementary math functions

import numpy as np  # numerical arrays
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string
print("pandas:", pd.__version__)  # pandas version string
print("matplotlib:", mpl.__version__)  # matplotlib version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline plots


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")


CODE_DIR = Path("..") / "code"  # companion module directory
sys.path.insert(0, str(CODE_DIR.resolve()))  # prefer local modules


### 2. Calibrated BCC parameters

We reuse the BCC parameters calibrated in Chapter 11. If a calibration
file is present, the parameter dictionary is loaded from disk; otherwise
a stylized, book-consistent set is returned.


In [ ]:
from dawp_pIIIb_ch11_bcc_calibration import load_bcc_parameters

params = load_bcc_parameters()
params


### 3. Simulating BCC short rates, variance, and index levels

We next generate sample paths for the short rate, variance, and index
levels under the calibrated BCC dynamics. The resulting paths provide
diagnostic plots and form the basis for Monte Carlo valuation.


In [ ]:
from dawp_pIIIb_ch12_bcc_mcs import simulate_bcc_paths

T = 1.0  # horizon in years
n_steps = 50  # time steps
n_paths = 10_000  # number of simulated paths

S, r, v, dt = simulate_bcc_paths(
    S0=params["S0"],
    r0=params["r0"],
    kappa_r=params["kappa_r"],
    theta_r=params["theta_r"],
    sigma_r=params["sigma_r"],
    kappa_v=params["kappa_v"],
    theta_v=params["theta_v"],
    sigma_v=params["sigma_v"],
    rho=params["rho"],
    v0=params["v0"],
    lamb=params["lamb"],
    mu_J=params["mu_J"],
    delta=params["delta"],
    T=T,
    M=n_steps,
    I=n_paths,
    anti_paths=True,
    moment_matching=True,
    seed=10,
)

t_grid = np.linspace(0.0, T, n_steps + 1)

fig, axes = plt.subplots(3, 1, sharex=True, figsize=(10.0, 8.0))

axes[0].plot(t_grid, r[:, :10])
axes[0].set_ylabel("short rate")
axes[0].set_title("Short-rate paths (CIR)")

axes[1].plot(t_grid, np.sqrt(v[:, :10]))
axes[1].set_ylabel("volatility")
axes[1].set_title("Volatility paths (sqrt variance)")

axes[2].plot(t_grid, S[:, :10])
axes[2].set_xlabel("time (years)")
axes[2].set_ylabel("index level")
axes[2].set_title("Index level paths (Bates)")

fig.tight_layout()
maybe_save(fig, "dawp_pIIIb_fig02_bcc_paths_overview")


### 4. European call valuation: Fourier vs. Monte Carlo

We now compare European call prices from the transform-based BCC
valuation routine to Monte Carlo estimates based on the simulation
engine. Agreement across strikes and maturities validates both
approaches.


In [ ]:
from dawp_pIIIa_ch09_bcc_option_valuation import bcc_call_value_int, cir_implied_short_rate
from dawp_pIIIb_ch12_bcc_mcs import european_call_mcs

t_list = [1.0 / 12.0, 0.5, 1.0, 2.0]  # maturities in years
k_list = [3050.0, params['S0'], 3400.0]  # strikes around the index level

rows = []
for T in t_list:
    r_eff = cir_implied_short_rate(
        params['r0'],
        params['kappa_r'],
        params['theta_r'],
        params['sigma_r'],
        T,
    )
    for K in k_list:
        C_fft = bcc_call_value_int(
            params['S0'],
            K,
            T,
            r_eff,
            params['kappa_v'],
            params['theta_v'],
            params['sigma_v'],
            params['rho'],
            params['v0'],
            params['lamb'],
            params['mu_J'],
            params['delta'],
        )
        C_mcs = european_call_mcs(
            params,
            K,
            T,
            n_steps=150,
            n_paths=150_000,
            anti_paths=False,
            moment_matching=True,
            seed=20,
        )
        diff = C_mcs - C_fft
        rows.append(
            {
                'T': T,
                'K': K,
                'C_fft': C_fft,
                'C_mcs': C_mcs,
                'diff': diff,
            }
        )

euro_comp = pd.DataFrame(rows)
euro_comp.round(4)


### 4a. Visual comparison of Fourier and Monte Carlo prices

We visualize the transform-based and Monte Carlo European call prices for each maturity
on separate panels to see the agreement across strikes.


In [ ]:
fig, axes = plt.subplots(len(t_list), 1, sharex=True, figsize=(8.0, 2.5 * len(t_list)))

if len(t_list) == 1:
    axes = [axes]

for ax, T in zip(axes, t_list):
    sub = euro_comp[euro_comp['T'] == T].sort_values('K')
    ax.plot(sub['K'], sub['C_fft'], 'o-', label='Fourier (BCC)')
    ax.plot(sub['K'], sub['C_mcs'], 'x--', label='Monte Carlo')
    ax.set_ylabel('call value')
    ax.set_title(f'T = {T:.2f} years')

axes[-1].set_xlabel('strike K')
axes[-1].legend(loc=0)
fig.tight_layout()
maybe_save(fig, 'dawp_pIIIb_fig02b_bcc_euro_fft_vs_mcs')


### 5. American put valuation via Least-Squares Monte Carlo

Finally we value an at-the-money American put under the calibrated BCC
dynamics using the Least-Squares Monte Carlo algorithm. The resulting
value and exercise statistics are inputs to the dynamic hedging
experiments in Chapter 13.


In [ ]:
from dawp_pIIIb_ch12_bcc_mcs import american_put_lsm

K = params["S0"]  # at-the-money strike
T = 1.0  # maturity in years

V0, S_mc, r_mc, v_mc, ex_mc, rg_mc, h_mc, dt_mc = american_put_lsm(
    params,
    K=K,
    T=T,
    n_steps=100,
    n_paths=100_000,
    basis_degree=3,
    anti_paths=True,
    moment_matching=True,
    seed=30,
)

print("American put value (LSM, BCC):", V0)
print(
    "Average exercise dates per path:",
    float(ex_mc.sum(axis=0).mean()),
)


### 5a. Exercise time distribution for American puts

We summarize when early exercise typically occurs by plotting the
distribution of exercise times implied by the LSM algorithm.


In [ ]:
exercise_step = ex_mc.argmax(axis=0)  # first exercise step per path
exercise_step = exercise_step[exercise_step > 0]  # ignore never-exercised
times = exercise_step * dt_mc

fig, ax = plt.subplots()
ax.hist(times, bins=20)
ax.set_xlabel('exercise time (years)')
ax.set_ylabel('frequency')
ax.set_title('Exercise time distribution (American put, BCC LSM)')
fig.tight_layout()
maybe_save(fig, 'dawp_pIIIb_fig02c_bcc_american_exercise_times')


<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">
